In [1]:
import rdflib
from SPARQLWrapper import SPARQLWrapper
from jsonpath_ng import jsonpath
import pandas as pd
from io import BytesIO
from IPython.display import display, HTML

# Querying the Knowledge Graph

Testing the KG and ontology with SPARQL built from the requirements and competency questions.


|Identifier|Requirement or Competency Question                                                                                           |
|---------|-----------------------------------------------------------------------------------------------------------------------------|
|[R1](#R1:-Articles-have-title,-year,-journal,-issue,-abstract,-volume-and-author)           |Articles have title, year, journal, issue, abstract, volume and author                                                       |
|[R2](#r2-each-article-may-refer-to-one-or-more-experiments)           |Each article may refer to one or more experiments                                                                            |
|[R3](#r3-each-experiment-must-have-a-catalyst)           |Each experiment must have a catalyst                                                                                         |
|[R4](#r4-each-experiment-may-have-additional-inputs-apart-from-the-catalyst)           |Each experiment may have additional inputs apart from the catalyst                                                           |
|[R5](#r5-each-experiment-must-specify-surface-area-and-band-gap)           |Each experiment must specify surface area and band gap                                                                       |
|[R6](#r6-an-experiment-must-indicate-operation-conditions-including-temperature-pressure-reaction-time-medium-and-operation-mode)           |An experiment must indicate operation conditions, including temperature, pressure, reaction time, medium and operation mode  |
|[R7](#r7-experiments-may-also-indicate-type-of-reactor)           |Experiments may also indicate type of reactor                                                                 |
|[R8](#r8-electrocatalysis-and-photoelectrocatalysis-experiments-may-also-indicate-electrochemical-configuration)          |Electrocatalysis and Photoelectrocatalysis experiments may also indicate electrochemical configuration                       |
|[R9](#r9-photocatalysis-and-photoelectrocatalysis-experiments-must-indicate-the-light-source-used-including-type-of-light-lamp-wavelength-irradiance-illuminated-area-and-power)           |Photocatalysis and Photoelectrocatalysis experiments must indicate the light source used, including type of light, lamp, wavelength, irradiance and power|
|[R10](#r9-photocatalysis-and-photoelectrocatalysis-experiments-must-indicate-the-light-source-used-including-type-of-light-lamp-wavelength-irradiance-illuminated-area-and-power)         |TiO2 catalysts must indicate crystal structure                                                                               |
|[R11](#r11-medium-reaction-must-be-either-gas-or-liquid)          |Medium reaction must be either gas or liquid                                                                                 |
|[R12](#r12-when-reaction-medium-is-liquid-the-ph-value-must-be-indicated)         |When reaction medium is liquid, the pH value must be indicated                                                               |
|[R13](#r13-operation-modes-must-be-either-batch-or-continuous)         |Operation modes must be either batch or continuous                                                                           |
|[R14](#r14-when-the-operation-mode-is-continuous-the-spacial-speed-value-must-be-indicated)         |When the operation mode is continuous, the spacial speed value must be indicated                                             |
|[R15](#r15-each-experiment-may-have-one-or-more-outputs)         |Each experiment may have one or more outputs                                                                                 |
|[R16](#r16-when-the-light-source-is-not-solar-or-solar-simulation-the-wavelength-must-be-indicated)         |When the light source is not solar or solar simulation, the wavelength must be indicated                                     |
|[R17](#r17-an-experiment-may-provide-conversion-metrics-quantum-efficiency-metrics-and-electrochemical-metrics)         |An experiment may provide conversion metrics, quantum efficiency metrics and electrochemical metrics                         |
|[CQ1](#cq1-how-many-experiments-are-reported-per-year)        |How many experiments are reported per year?                                                                                  |
|[CQ2](#cq2-how-many-experiments-are-reported-per-country)       |How many experiments are reported per country?                                                                               |
|[CQ3](#cq3-how-many-experiments-are-there-per-type-of-catalyst)       |How many experiments are there per type of catalyst?                                                                         |
|[CQ4](#cq4-which-articles-have-been-published-in-the-acs-nano-journal-in-2020-in-volume-14)       |Which articles have been published in the ACS nano Journal in 2020 in volume 14?                                            |
|[CQ5](#cq5-which-experiments-use-tio2-as-catalyst-liquid-medium-an-produce-h2-as-output)       |Which experiments use TiO2 as catalyst, liquid medium an produce H2 as output?                                               |
|[CQ6](#cq6-which-experiments-use-tio2-as-catalyst-liquid-medium-and-produce-the-most-h2-output-in-µmolgh)       |Which experiments use TiO2 as catalyst, liquid medium and produce the most H2 output in µmol/gh?                             |


### Config to query the KG online

In [2]:
sparql = SPARQLWrapper(
    "https://graphdb.linkeddata.es/repositories/SolarChem"
)

prefixes = """
PREFIX bibo: <http://purl.org/ontology/bibo/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX solar: <https://w3id.org/solar/o/core#>
PREFIX solarec: <https://w3id.org/solar/o/ec#>
PREFIX solarpec: <https://w3id.org/solar/o/pec#>
PREFIX solarpc: <https://w3id.org/solar/o/pc#>
PREFIX schema: <https://schema.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX chebi: <http://purl.obolibrary.org/obo/CHEBI_> 
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX iao: <http://purl.obolibrary.org/obo/IAO_> 
PREFIX obi: <http://purl.obolibrary.org/obo/OBI_> 
PREFIX unit: <http://qudt.org/vocab/unit/>

"""

def run_query_csv(query):
    sparql.setReturnFormat('csv')
    sparql.setQuery(query)
    try:
        results = sparql.queryAndConvert()
        #decode_results= results.decode(encoding='utf-8', errors='strict')
        df = pd.read_csv(BytesIO(results), sep=",")
        display(HTML(df.to_html()))
    except Exception as e:
        print(e)

def run_query_json(query):
    sparql.setReturnFormat('json')
    sparql.setQuery(query)
    try:
        results = sparql.queryAndConvert()
        return(results)
    except Exception as e:
        print(e)

## R1: Articles have title, year, journal, issue, abstract, volume and author

In [45]:
query = prefixes + """
    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a schema:ScholarlyArticle ;
            bibo:doi ?doi ;
            schema:name ?name ;
            schema:volumeNumber ?volume ;
            schema:issueNumber ?issue ;
            schema:datePublished ?date ;
            schema:pageStart ?pagestart;
            schema:pageEnd ?pageend;
            schema:isPartOf ?journal .
    } 
"""

q_res = run_query_json(query)

for row in q_res['results']['bindings']:
    print(f"Number of articles with all properties: {row['count']['value']}")

Number of articles with all properties: 1254


## R2: Each article may refer to one or more experiments

In [46]:
query = prefixes + """
    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a schema:ScholarlyArticle ;
            solar:hasExperimentExecution ?expexec .
    }
"""

q_res = run_query_json(query)

for row in q_res['results']['bindings']:
    print(f"Number of articles with experiments associated: {row['count']['value']}")

Number of articles with experiments associated: 1093


## R3: Each experiment must have a catalyst

In [50]:
query = prefixes + """
    SELECT ?expexec  ?catalyst ?chem
    WHERE {
        ?expexec prov:used ?input .
        ?input a solar:Input ;
            rdfs:label ?catalyst ;
            solar:hasRole solar:Catalyst ;
            solar:hasChemical ?chem
    } LIMIT 10

"""

q_res = run_query_csv(query)

,expexec,catalyst,chem
0,https://w3id.org/solar/i/ExpExec/6517,TiO2,http://www.wikidata.org/entity/Q193521
1,https://w3id.org/solar/i/ExpExec/3089,TiO2,http://www.wikidata.org/entity/Q193521
2,https://w3id.org/solar/i/ExpExec/2448,TiO2,http://www.wikidata.org/entity/Q193521
3,https://w3id.org/solar/i/ExpExec/1888,TiO2,http://www.wikidata.org/entity/Q193521
4,https://w3id.org/solar/i/ExpExec/5600,TiO2,http://www.wikidata.org/entity/Q193521
5,https://w3id.org/solar/i/ExpExec/2732,TiO2,http://www.wikidata.org/entity/Q193521
6,https://w3id.org/solar/i/ExpExec/5245,TiO2,http://www.wikidata.org/entity/Q193521
7,https://w3id.org/solar/i/ExpExec/526,TiO2,http://www.wikidata.org/entity/Q193521
8,https://w3id.org/solar/i/ExpExec/94,TiO2,http://www.wikidata.org/entity/Q193521
9,https://w3id.org/solar/i/ExpExec/63,TiO2,http://www.wikidata.org/entity/Q193521


## R4: Each experiment may have additional inputs apart from the catalyst

In [51]:
query = prefixes + """
    SELECT DISTINCT  ?expexec ?inputlabel ?role
    WHERE {
        ?expexec prov:used ?input .
        ?input a solar:Input ;
            rdfs:label ?inputlabel ;
            solar:hasRole ?role .
        FILTER(?role != solar:Catalyst)
    } LIMIT 10

"""

q_res = run_query_csv(query)

,expexec,inputlabel,role
0,https://w3id.org/solar/i/ExpExec/3745,CuAlGaO4,https://w3id.org/solar/o/core#Co-catalyst
1,https://w3id.org/solar/i/ExpExec/3745,H2,https://w3id.org/solar/o/pc#Reductant
2,https://w3id.org/solar/i/ExpExec/347,H2O,https://w3id.org/solar/o/pc#Reductant
3,https://w3id.org/solar/i/ExpExec/7174,Ag,https://w3id.org/solar/o/core#Dopant
4,https://w3id.org/solar/i/ExpExec/7174,ZnO,https://w3id.org/solar/o/core#Co-catalyst
5,https://w3id.org/solar/i/ExpExec/7174,H2O,https://w3id.org/solar/o/pc#Reductant
6,https://w3id.org/solar/i/ExpExec/6294,NiFe-LDH,https://w3id.org/solar/o/core#Co-catalyst
7,https://w3id.org/solar/i/ExpExec/6294,TEOA,https://w3id.org/solar/o/pc#Reductant
8,https://w3id.org/solar/i/ExpExec/1750,H2O,https://w3id.org/solar/o/pc#Reductant
9,https://w3id.org/solar/i/ExpExec/6821,TEOA,https://w3id.org/solar/o/pc#Reductant


## R5: Each experiment must specify surface area and band gap

In [52]:
query = prefixes + """
    SELECT DISTINCT ?expexec ?surfacearea ?saunit  ?bgap ?bgapunit
    WHERE {
        ?expexec prov:used ?input .
        ?input a solar:Input ;
            solar:hasBandGap [ qudt:numericValue ?bgap; qudt:unit ?bgapunit ] ;
            solar:hasSurfaceArea [ qudt:numericValue ?surfacearea; qudt:unit ?saunit ] .
    } LIMIT 10

"""

q_res = run_query_csv(query)

,expexec,surfacearea,saunit,bgap,bgapunit
0,https://w3id.org/solar/i/ExpExec/3089,254.10,http://qudt.org/vocab/unit/M2-PER-GM,2.81,http://qudt.org/vocab/unit/EV
1,https://w3id.org/solar/i/ExpExec/2732,51.00,http://qudt.org/vocab/unit/M2-PER-GM,3.32,http://qudt.org/vocab/unit/EV
2,https://w3id.org/solar/i/ExpExec/63,67.60,http://qudt.org/vocab/unit/M2-PER-GM,2.98,http://qudt.org/vocab/unit/EV
3,https://w3id.org/solar/i/ExpExec/67,79.70,http://qudt.org/vocab/unit/M2-PER-GM,2.74,http://qudt.org/vocab/unit/EV
4,https://w3id.org/solar/i/ExpExec/2644,14.51,http://qudt.org/vocab/unit/M2-PER-GM,3.20,http://qudt.org/vocab/unit/EV
5,https://w3id.org/solar/i/ExpExec/403,39.79,http://qudt.org/vocab/unit/M2-PER-GM,3.06,http://qudt.org/vocab/unit/EV
6,https://w3id.org/solar/i/ExpExec/4719,67.00,http://qudt.org/vocab/unit/M2-PER-GM,3.36,http://qudt.org/vocab/unit/EV
7,https://w3id.org/solar/i/ExpExec/105,39.40,http://qudt.org/vocab/unit/M2-PER-GM,2.95,http://qudt.org/vocab/unit/EV
8,https://w3id.org/solar/i/ExpExec/1634,84.00,http://qudt.org/vocab/unit/M2-PER-GM,2.87,http://qudt.org/vocab/unit/EV
9,https://w3id.org/solar/i/ExpExec/467,156.34,http://qudt.org/vocab/unit/M2-PER-GM,3.32,http://qudt.org/vocab/unit/EV


## R6: An experiment must indicate operation conditions, including temperature, pressure, reaction time, medium and operation mode
Note: Medium and operation mode classes in onto, not in this KG's version

In [53]:
query = prefixes + """
    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition ?condition .
        ?condition a ?condition_type .
        VALUES ?condition_type {solar:TimeCondition solar:PressureCondition solar:TemperatureCondition}
    }
"""

q_res = run_query_json(query)

for row in q_res['results']['bindings']:
    print(f"Number of experiments with conditions: {row['count']['value']}")

Number of experiments with conditions: 6664


## R7: Experiments may also indicate type of reactor

In [55]:
query = prefixes + """
    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition ?reactorCondition .
        ?reactorCondition a solar:ReactorCondition
    }
"""

q_res = run_query_json(query)

for row in q_res['results']['bindings']:
   print(f"Number of articles with reactor conditions: {row['count']['value']}")

Number of articles with reactor conditions: 6664


## R8: Electrocatalysis and Photoelectrocatalysis experiments may also indicate electrochemical configuration
No experiments so far with this condition, only from photocatalysis

In [56]:
query = prefixes + """
    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition ?condition .
        ?condition a solar:ElectrochemicalConfiguration .
    }
"""

q_res = run_query_json(query)

for row in q_res['results']['bindings']:
    print(f"Number of articles with electrochemical configuration: {row['count']['value']}")

Number of articles with electrochemical configuration: 0


## R9: Photocatalysis and Photoelectrocatalysis experiments must indicate the light source used, including type of light, lamp, wavelength, irradiance, illuminated area and power

In [59]:
query = prefixes + """
    SELECT DISTINCT ?expexec ?lamp ?wl ?irradiance ?power ?illum_area
    WHERE {
        ?expexec solar:hasLightSource ?light_source.
        ?light_source a obi:0400065 ;
            solar:hasLamp ?lamp ;
            solar:hasWavelength ?wl ;
            solar:hasIrradiance [qudt:numericValue ?irradiance] ;
            solar:hasPower [qudt:numericValue ?power] ;
            solar:hasIlluminatedArea [qudt:numericValue ?illum_area] .
            
    } LIMIT 5
"""

q_res = run_query_csv(query)

,expexec,lamp,wl,irradiance,power,illum_area
0,https://w3id.org/solar/i/ExpExec/2939,https://w3id.org/solar/o/core#MercuryLamp,https://w3id.org/solar/o/core#192-280WL,5.0,8.0,1.60
1,https://w3id.org/solar/i/ExpExec/3753,https://w3id.org/solar/o/core#XenonLamp,https://w3id.org/solar/o/core#192-280WL,2700.0,300.0,368.00
2,https://w3id.org/solar/i/ExpExec/499,https://w3id.org/solar/o/core#XenonLamp,https://w3id.org/solar/o/core#192-280WL,348.0,350.0,0.01
3,https://w3id.org/solar/i/ExpExec/436,https://w3id.org/solar/o/core#XenonLamp,https://w3id.org/solar/o/core#400-780WL,1.2,500.0,0.05
4,https://w3id.org/solar/i/ExpExec/6683,https://w3id.org/solar/o/core#MercuryLamp,https://w3id.org/solar/o/core#192-280WL,20.0,350.0,0.01


## R10: TiO2 catalysts must indicate crystal structure

In [60]:
query = prefixes + """
    SELECT ?input ?crystal_str
    WHERE {
        ?input a solar:Input ;
            rdfs:label "TiO2" ;
            solar:hasRole solar:Catalyst ;
            solar:crystalStructure ?crystal_str .
    } LIMIT 10

"""

q_res = run_query_csv(query)

,input,crystal_str
0,https://w3id.org/solar/i/Input/Catalyst/6517-TiO2,Anatase
1,https://w3id.org/solar/i/Input/Catalyst/3089-TiO2,Anatase
2,https://w3id.org/solar/i/Input/Catalyst/2448-TiO2,Anatase
3,https://w3id.org/solar/i/Input/Catalyst/1888-TiO2,Anatase
4,https://w3id.org/solar/i/Input/Catalyst/5600-TiO2,Mix
5,https://w3id.org/solar/i/Input/Catalyst/2732-TiO2,Rutile
6,https://w3id.org/solar/i/Input/Catalyst/5245-TiO2,Mix
7,https://w3id.org/solar/i/Input/Catalyst/526-TiO2,P25
8,https://w3id.org/solar/i/Input/Catalyst/94-TiO2,P25
9,https://w3id.org/solar/i/Input/Catalyst/63-TiO2,Anatase


## R11: Medium reaction must be either gas or liquid

In [61]:
query = prefixes + """
    SELECT DISTINCT ?label
    WHERE {
        ?medium a solar:ReactionMediumCondition ;
                rdfs:label ?label
    }
"""

q_res = run_query_csv(query)

,label
0,Liquid Medium
1,Gas Medium


## R12: When reaction medium is liquid, the pH value must be indicated

In [62]:
query = prefixes + """
    SELECT ?expexec ?ph
    WHERE {
        ?expexec solar:hasCondition solar:LiquidMedium ;
            solar:hasCondition ?ph_cond .
        ?ph_cond a solar:pHCondition ;
            qudt:numericValue ?ph .
    } LIMIT 10
"""

q_res = run_query_csv(query)

,expexec,ph
0,https://w3id.org/solar/i/ExpExec/6208,6
1,https://w3id.org/solar/i/ExpExec/3074,11
2,https://w3id.org/solar/i/ExpExec/4882,13
3,https://w3id.org/solar/i/ExpExec/3071,3
4,https://w3id.org/solar/i/ExpExec/6161,8
5,https://w3id.org/solar/i/ExpExec/3075,1
6,https://w3id.org/solar/i/ExpExec/5137,7
7,https://w3id.org/solar/i/ExpExec/5062,7
8,https://w3id.org/solar/i/ExpExec/5171,7
9,https://w3id.org/solar/i/ExpExec/6257,7


## R13: Operation modes must be either batch or continuous

In [63]:
query = prefixes + """
    SELECT DISTINCT ?label
    WHERE {
        ?medium a solar:OperationModeCondition ;
                rdfs:label ?label
    }
"""

q_res = run_query_csv(query)

,label
0,Batch Mode
1,Continuous Mode


## R14: When the operation mode is continuous, the spacial speed value must be indicated


In [64]:
query = prefixes + """
    SELECT ?expexec ?restime ?unit
    WHERE {
        ?expexec solar:hasCondition solar:ContinuousMode ;
            solar:hasCondition ?restime_cond .
        ?restime_cond a solar:ResidenceTimeCondition ;
            qudt:numericValue ?restime;
            qudt:unit ?unit.
    } LIMIT 5
"""

q_res = run_query_csv(query)

,expexec,restime,unit
0,https://w3id.org/solar/i/ExpExec/4921,0.24,http://qudt.org/vocab/unit/PER-MIN
1,https://w3id.org/solar/i/ExpExec/149,83.30,http://qudt.org/vocab/unit/PER-MIN
2,https://w3id.org/solar/i/ExpExec/4919,0.24,http://qudt.org/vocab/unit/PER-MIN
3,https://w3id.org/solar/i/ExpExec/687,15.00,http://qudt.org/vocab/unit/PER-MIN
4,https://w3id.org/solar/i/ExpExec/3770,39.00,http://qudt.org/vocab/unit/PER-MIN


## R15: Each experiment may have one or more outputs

In [65]:
query = prefixes + """
    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?output a solar:Output ;
            prov:wasGeneratedBy ?expexec .
    }

"""

q_res = run_query_json(query)

for row in q_res['results']['bindings']:
    print(f"Number of experiments with outputs: {row['count']['value']}")

Number of experiments with outputs: 6664


## R16: When the light source is not solar or solar simulation, the wavelength must be indicated

In [66]:
query = prefixes + """
    SELECT (COUNT (DISTINCT ?light_source) AS ?count)
    WHERE {
        ?light_source a ?type ;
            solar:hasWavelength ?wl .
        FILTER (?type != solar:SolarLightSource || ?type != solar:SolarSimulatorLightSource )
    } #LIMIT 10
"""

q_res = run_query_json(query)

for row in q_res['results']['bindings']:
    print(f"Number of lightsources with all properties: {row['count']['value']}")

Number of lightsources with all properties: 5398


## R17: An experiment may provide conversion metrics, quantum efficiency metrics and electrochemical metrics
Not yet in the KG, only photocatalysis experiments

In [9]:
query = prefixes + """
   SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?output a solar:Output ;
            prov:wasGeneratedBy ?expexec ;
            solar:hasConversionmetric ?cmetric ;
            solar:hasQuantumEfficiencyMetric ?qemetric ;
            solar:hasElectrochemicalMetric ?emetric .
    } 
"""

q_res = run_query_csv(query)

,count
0,0


## CQ1: How many experiments are reported per year?

In [67]:
query = prefixes + """
   SELECT ?year (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?article a schema:ScholarlyArticle ;
            schema:datePublished ?year ;
            solar:hasExperimentExecution ?expexec
    } GROUP BY ?year
    ORDER BY DESC (?year)

"""

q_res = run_query_csv(query)

,year,count
0,2022,263
1,2021,1176
2,2020,1149
3,2019,311
4,2018,530
5,2017,719
6,2016,343
7,2015,579
8,2014,392
9,2013,347


## CQ2: How many experiments are reported per country?
Check, results are weird


In [68]:
query = prefixes + """
    SELECT ?country (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?article a schema:ScholarlyArticle ;
                solar:hasExperimentExecution ?expexec ;
                schema:author ?author .
        ?author schema:affiliation ?affiliation .
        ?affiliation schema:addressCountry ?country
    } GROUP BY ?country
    ORDER BY DESC (?count)

"""

q_res = run_query_csv(query)

,country,count
0,https://w3id.org/solar/i/Country/CN,6086
1,https://w3id.org/solar/i/Country/JP,5051
2,https://w3id.org/solar/i/Country/ES,4973
3,https://w3id.org/solar/i/Country/AU,4970
4,https://w3id.org/solar/i/Country/SA,4947
5,https://w3id.org/solar/i/Country/US,4945
6,https://w3id.org/solar/i/Country/MY,4933
7,https://w3id.org/solar/i/Country/SG,4903
8,https://w3id.org/solar/i/Country/DE,4870
9,https://w3id.org/solar/i/Country/PK,4862


## CQ3: How many experiments are there per type of catalyst?

In [69]:
query = prefixes + """
    SELECT ?catalyst (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec prov:used ?input .
        ?input solar:hasRole solar:Catalyst ;
            rdfs:label ?catalyst .
    } GROUP BY ?catalyst
    ORDER BY DESC (?count)

"""

q_res = run_query_csv(query)

,catalyst,count
0,TiO2,2131
1,g-C3N4,745
2,Ti-KIT-6,201
3,ZnO,182
4,CeO2,160
5,ZnS,138
6,CdS,112
7,BiOCl,85
8,Cu2O,83
9,Bi2WO6,82


## CQ4: Which articles have been published in the ACS nano Journal in 2020 in volume 14?

In [70]:
query = prefixes + """
    SELECT ?article ?title ?doi
    WHERE {
        ?article a schema:ScholarlyArticle ;
            schema:name ?title ;
            bibo:doi ?doi ;
            schema:volumeNumber "14" ;
            schema:datePublished "2020"^^xsd:gYear ;
            schema:isPartOf "ACS nano" .
    } 
"""

q_res = run_query_csv(query)

,article,title,doi
0,https://w3id.org/solar/i/Article/911,Direct Z-Scheme Heterojunction of Semicoherent FAPbBr<sub>3</sub>/Bi<sub>2</sub>WO<sub>6</sub> Interface for Photoredox Reaction with Large Driving Force,https://doi.org/10.1021/acsnano.0c03146
1,https://w3id.org/solar/i/Article/944,A Single Cu-Center Containing Enzyme-Mimic Enabling Full Photosynthesis under CO<sub>2</sub> Reduction,https://doi.org/10.1021/acsnano.0c02940
2,https://w3id.org/solar/i/Article/1012,Identification of Halogen-Associated Active Sites on Bismuth-Based Perovskite Quantum Dots for Efficient and Selective CO<sub>2</sub>-to-CO Photoreduction,https://doi.org/10.1021/acsnano.0c04659
3,https://w3id.org/solar/i/Article/1031,Rare-Earth Single-Atom La–N Charge-Transfer Bridge on Carbon Nitride for Highly Efficient and Selective Photocatalytic CO<sub>2</sub> Reduction,https://doi.org/10.1021/acsnano.0c07083
4,https://w3id.org/solar/i/Article/894,Room-Temperature Synthesis of Single Iron Site by Electrofiltration for Photoreduction of CO<sub>2</sub> into Tunable Syngas,https://doi.org/10.1021/acsnano.0c02162


## CQ5: Which experiments use TiO2 as catalyst, liquid medium an produce H2 as output?

In [75]:
query = prefixes + """
    SELECT (COUNT (DISTINCT?expexec) AS ?count)
    WHERE {
        ?expexec prov:used ?input ;
            solar:hasCondition solar:LiquidMedium .
            
        ?input solar:hasRole solar:Catalyst ;
            rdfs:label "TiO2" .
    
        ?output rdfs:label "H2" ;
            prov:wasGeneratedBy ?expexec .
    }
"""

q_res = run_query_json(query)

for row in q_res['results']['bindings']:
    print(f"Experiments: {row['count']['value']}")

Experiments: 896


## CQ6: Which experiments use TiO2 as catalyst, liquid medium and produce the most H2 output in µmol/gh?
Slow with RDFLib, works fine in GraphDB

In [79]:
query = prefixes + """
    SELECT ?expexec ?value
        WHERE {
            ?expexec prov:used ?input ;
                solar:hasCondition solar:LiquidMedium .
                
            ?input solar:hasRole solar:Catalyst ;
                rdfs:label "TiO2" .
        
            ?output rdfs:label "H2" ;
                qudt:unit unit:MicroMOL-PER-GM-HR ;
                prov:wasGeneratedBy ?expexec ;
                qudt:numericValue ?value .
    } ORDER BY DESC(?value)
    LIMIT 10
"""

q_res = run_query_csv(query)

,expexec,value
0,https://w3id.org/solar/i/ExpExec/1718,6111.11
1,https://w3id.org/solar/i/ExpExec/4718,4000.00
2,https://w3id.org/solar/i/ExpExec/5846,2250.00
3,https://w3id.org/solar/i/ExpExec/4720,2018.00
4,https://w3id.org/solar/i/ExpExec/4719,1613.00
5,https://w3id.org/solar/i/ExpExec/2317,1406.25
6,https://w3id.org/solar/i/ExpExec/1719,833.33
7,https://w3id.org/solar/i/ExpExec/7187,360.00
8,https://w3id.org/solar/i/ExpExec/2318,312.50
9,https://w3id.org/solar/i/ExpExec/2468,310.00


------------
# Graphics

## Type of catalyst vs publication date
Two catalysts selected only, could be expanded

In [80]:
query = prefixes + """
    SELECT ?year (COUNT (DISTINCT ?input1) AS ?TiO2) (COUNT (DISTINCT ?input2) AS ?gC3N4)
    WHERE {
        {?article1 a schema:ScholarlyArticle ;
            schema:datePublished ?year ;
            solar:hasExperimentExecution ?expexec1 .
       ?expexec1 prov:used ?input1 .
       ?input1 solar:hasRole solar:Catalyst ;
            rdfs:label "TiO2" .}
        UNION
        {?article2 a schema:ScholarlyArticle ;
            schema:datePublished ?year ;
            solar:hasExperimentExecution ?expexec2 .
        ?expexec2 prov:used ?input2 .
        ?input2 solar:hasRole solar:Catalyst ;
            rdfs:label "g-C3N4" .}
 
    } GROUP BY ?year
    ORDER BY DESC (?year)
"""

q_res = run_query_csv(query)

,year,TiO2,gC3N4
0,2022,55,38
1,2021,145,164
2,2020,196,208
3,2019,56,46
4,2018,225,47
5,2017,334,53
6,2016,121,98
7,2015,177,26
8,2014,210,54
9,2013,153,11
